In [ ]:
import sys
sys.path.append("../")
from aoc_utils import *

# Home

Each day's work will consist of three tasks:
- **Input**: Parse the day's input file with the function `parse(day, parser, sep)`, which treats the input as a sequence of *entries*, separated by `sep` (default newline); applies `parser` to each entry; and returns the results as a tuple. (Note: `ints` and `atoms` are useful `parser` functions (as are `int` and `str`).)
- **Part 1**: Write code to compute the answer to Part 1, and submit the answer to the AoC site. Use the function `answer` to record the correct answer and serve as a regression test when I re-run the notebook.
- **Part 2**: Repeat coding and `answer` for Part 2.

1. [Day 1](#day-1)
2. [Day 2](#day-2)
3. [Day 3](#day-3)
4. [Day 4](#day-4)
5. [Day 5](#day-5)
6. [Day 6](#day-6)
7. [Day 7](#day-7)


[home](#home)
# Day 1
[Calorie Counting](https://adventofcode.com/2022/day/1)

Find the Elf carrying the most Calories.  
How many total Calories is that Elf carrying ?


In [ ]:
get_in_file(1,2022)
in1 = parse(1, ints, sep='\n\n')

elves_food = [sum(elf_food) for elf_food in in1]
max_food_cal = max(elves_food)
max_food_cal

In [ ]:
submit(max_food_cal, part="a", day=1, year=2022)

### Part 2
Find the top three Elves carrying the most Calories.<br>
How many Calories are those Elves carrying in total ?

In [ ]:
top_three = sorted(elves_food)[-3:]
top_three

In [ ]:
submit(sum(top_three), part="b", day=1, year=2022)

[home](#home)
# Day 2
[Rock Paper Scissors](https://adventofcode.com/2022/day/2)

Win Rock Paper Scissors tournament  
What would your total score be if everything goes exactly according to your strategy guide?

The first column is what your opponent is going to play:
- A for Rock
- B for Paper
- C for Scissors

The second column is what you should play in response:
- X Rock
- Y Paper
- Z Scissors

In [ ]:
get_in_file(2,2022)
#B Y
#A Z
#...
#line_parser = lambda x: "".join(str.split(x))
line_parser = lambda x: tuple(str.split(x))
#line_parser = lambda x: str.strip(x)
in2 = Input(2, line_parser)
#(('B','Y'),('A','Z')...)

The score for a single round is the sum of :
- score of your shape :
    - 1 for Rock
    - 2 for Paper
    - 3 for Scissors
- score of your outcome :
    - 0 lost
    - 3 draw
    - 6 win


In [ ]:
my_shape_score = {'X':1, 'Y':2, 'Z':3}
other_shape_score = {'A':1, 'B':2, 'C':3}
outcome_score = {'lost':0, 'draw':3, 'win':6}
comb = product('ABC','XYZ')
score_comb = [3,6,0,
              0,3,6,
              6,0,3]
outcome_look_up = dict(zip(comb,score_comb))
rounds = [ my_shape_score[x[1]] + outcome_look_up[x] for x in in2]
sum(rounds)

In [ ]:
submit(sum(rounds), part="a", day=2, year=2022)

### Part 2

Now the second column says how the round needs to end:
- X loose
- Y draw
- Z win

In [ ]:
outcome_score = {'X':0, 'Y':3, 'Z':6}
comb = product('ABC','XYZ')
score_comb = [3,1,2,
              1,2,3,
              2,3,1]
my_shape_look_up = dict(zip(comb,score_comb)) 
rounds = [ my_shape_look_up[x] + outcome_score[x[1]] for x in in2]

In [ ]:
submit(sum(rounds), part="b", day=2, year=2022)

[home](#home)
# Day 3
[Rucksack Reorganization](https://adventofcode.com/2022/day/3)

One Elf has the important job of loading all of the rucksacks with supplies for the jungle journey.  
Each rucksack has two large compartments. All items of a given type are meant to go into exactly one of the two compartments.  
The Elf that did the packing failed to follow this rule for exactly **one item** type per rucksack.  
Find the item type that appears in both compartments of each rucksack.  
What is the sum of the priorities of those item types?

In [ ]:
get_in_file(3,2022)
in3 = Input(3)
#in3 = Input(3, file_template='test/test{}.txt')

To help prioritize item rearrangement, every item type can be converted to a priority:

- Lowercase item types a through z have priorities 1 through 26.
- Uppercase item types A through Z have priorities 27 through 52.

In [ ]:
# priority map
prio = dict(zip(string.ascii_letters,range(1,len(string.ascii_letters)+1)))

def common_item(ln:str) -> chr :
    assert(len(ln)%2==0)
    # split in two
    st_half , nd_half = ln[:len(ln)//2],ln[len(ln)//2:]
    # intersection must give only one elem 
    return (set(st_half) & set(nd_half)).pop()

priority_items = mapt(lambda x : prio[x], mapt(common_item, in3))

In [ ]:
submit(sum(priority_items), part="a", day=3, year=2022)

### Part 2
For safety, the Elves are divided into groups of three.  
Every Elf carries a badge that identifies their group.  
For efficiency, within each group of three Elves, the badge is the **only item type carried by all three Elves**.  
The only way to tell which item type is the right one is by finding the one item type that is **common between all three Elves** in each group.  
Find the item type that corresponds to the badges of each three-Elf group.  
What is the sum of the priorities of those item types?

In [ ]:

priority_items = mapt(lambda y : prio[y.pop()], mapt(lambda x: reduce(set.intersection, [set(e) for e in x]), grouper(in3, 3, incomplete='strict')))

In [ ]:
submit(sum(priority_items), part="b", day=3, year=2022)

[home](#home)
# Day 4
[Camp Cleanup](https://adventofcode.com/2022/day/4)

In how many assignment pairs does one range fully contain the other?

In [ ]:
from typing import Tuple, List
day=4
get_in_file(day,2022)
# first split on ',' then split the iteritems on '-' 
line_parser = lambda x : mapt(lambda y : y.split('-'), str.rstrip(x).split(','))
in4 = Input(day, line_parser=line_parser)
#in4 = Input(day, line_parser=line_parser, file_template='test/test{}.txt')

def make_sets ( pairs : Tuple[List[chr]] ) -> Tuple[set] :
    st_pair, nd_pair = pairs
    x1, x2 = mapt(int, st_pair)
    st_set = set(range(x1,x2+1))
    y1, y2 = mapt(int, nd_pair)
    nd_set = set(range(y1,y2+1))
    return st_set,nd_set


In [ ]:
def fully_contain( pairs : Tuple[List[chr]] ) -> bool:
    st_set, nd_set = make_sets(pairs)
    return st_set.issubset(nd_set) or nd_set.issubset(st_set)

#Count how many times the predicate is true
quantify(in4, fully_contain)

In [ ]:
submit(quantify(in4, fully_contain), part="a", day=4, year=2022)

### Part 2

In how many assignment pairs do the ranges overlap?

In [ ]:
def overlap( pairs : Tuple[List[chr]] ) -> bool:
    st_set, nd_set = make_sets(pairs)
    return len(st_set.intersection(nd_set)) > 0

#Count how many times the predicate is true
quantify(in4, overlap)

In [ ]:
submit(quantify(in4, overlap), part="b", day=4, year=2022)

[home](#home)


# Day 5
[Supply Stacks](https://adventofcode.com/2022/day/5)

In [ ]:
day=5
get_in_file(day,2022)
# drawing of the starting stacks of crates
# rearrangement procedure  
line_parser = lambda x: x.split('\n\n')
start_stack, moves = parse(5, sep='\n\n')


### modify **start_stack** as dict  

In [ ]:
start_stack.split('\n')

In [ ]:
keys, *values = tuple(reversed(mapt(lambda x : tuple(compress(x, cycle((0,1,0,0)))), start_stack.split('\n'))))
values

In [ ]:
transpose_tuples = tuple(zip(*values))
transpose_tuples

In [ ]:
ss_dict = dict(zip(map(int, keys), mapt(lambda x: str.strip(''.join(x)), transpose_tuples)))
ss_dict

### .... put all together

In [ ]:
def top_crates_900x(start_stack:str, moves:str, part='a' ) -> str:
    keys, *values = tuple(reversed(mapt(lambda x : tuple(compress(x, cycle((0,1,0,0)))), start_stack.split('\n'))))
    transpose_tuples = tuple(zip(*values))
    ss_dict = dict(zip(map(int, keys), mapt(lambda x: str.strip(''.join(x)), transpose_tuples)))
    moves_list = moves.split('\n')
    for mv in mapt(ints, moves_list) :
        how_many, src, dest = mv
        if part == 'a':
            ss_dict[dest] = ss_dict[dest] + ss_dict[src][-how_many:][::-1]
        else:
            ss_dict[dest] = ss_dict[dest] + ss_dict[src][-how_many:]
        ss_dict[src] = ss_dict[src][:-how_many]

    pp.pprint(ss_dict)
    return ''.join(mapt(lambda x: x[-1:], ss_dict.values()))

In [ ]:
top_crates = top_crates_900x(start_stack, moves)

In [ ]:
submit(top_crates, part="a", day=5, year=2022)

### Part 2

The CrateMover 9001 is notable for many new and exciting features ... and the ability to pick up and move multiple crates at once

In [ ]:
top_crates = top_crates_900x(start_stack, moves, part='b')

In [ ]:
submit(top_crates, part="b", day=5, year=2022)

[home](#home)
# Day 6
[link](https://adventofcode.com/2022/day/6)